# 04-工具调用与结构化输出测试

## GLM-4-Flash-250414 模型能力

根据官方文档，GLM-4-Flash-250414 具备以下能力：

| 能力 | 支持情况 | 说明 |
|:---|:---:|:---|
| **流式输出** | ✅ 支持 | 实时流式响应 |
| **Function Call** | ✅ 支持 | 强大的工具调用能力 |
| **上下文缓存** | ✅ 支持 | 智能缓存机制，优化长对话 |
| **结构化输出** | ✅ 支持 | JSON 等结构化格式 |
| **MCP** | ✅ 支持 | 调用外部 MCP 工具 |
| **思考模式** | ❌ 不支持 | 该模型不支持深度思考 |

## 模型规格

| 规格项 | 数值 |
|:---|:---|
| 上下文窗口 | 128K |
| 多语言支持 | 26种语言 |
| 定位 | 智谱首个免费大模型 API |

## 功能特色

1. **超长上下文** - 128K 上下文，相当于 300 页书籍
2. **多语言支持** - 支持 26 种语言
3. **网页检索** - 支持外部工具调用，通过网络搜索获取信息

## ⚠️ 关键区别

与 GLM-4.7-Flash 的主要区别：

| 能力 | GLM-4-Flash-250414 | GLM-4.7-Flash |
|:---|:---:|:---:|
| 思考模式 | ❌ 不支持 | ✅ 支持 |
| 工具调用 | ✅ 支持 | ✅ 支持 |
| 结构化输出 | ✅ 支持 | ✅ 支持 |
| 上下文 | 128K | 200K |

## 测试内容
1. 环境配置
2. 基础对话测试
3. Function Call（工具调用）
4. 结构化输出（JSON）
5. 网页检索（Web Search）
6. 智能写作
7. 智能翻译
8. 实体抽取
9. 流式输出
10. 长上下文测试

## 1. 环境配置

In [ ]:
# 安装依赖
# !pip install openai python-dotenv jsonschema

In [ ]:
import os
import json
from openai import OpenAI

# 加载环境变量
try:
    from dotenv import load_dotenv
    load_dotenv()
    api_key = os.getenv('VITE_ZHIPU_API_KEY')
except:
    api_key = None

# 如果环境变量中没有，使用默认 API Key
if not api_key:
    api_key = "f7ec61a7856a4f4389e037eb588c69df.0pZgcFlTCk9GHCXM"

BASE_URL = "https://open.bigmodel.cn/api/paas/v4"

# 初始化客户端
client = OpenAI(
    api_key=api_key,
    base_url=BASE_URL
)

print("✅ 客户端初始化成功！")
print(f"模型：GLM-4-Flash-250414")
print(f"注意：该模型不支持思考模式，但支持工具调用和结构化输出")

## 2. 基础对话测试

In [ ]:
# 2.1 简单问候
print("💬 测试1：简单问候\n")

response = client.chat.completions.create(
    model="glm-4-flash-250414",
    messages=[
        {"role": "user", "content": "你好，请介绍一下自己"}
    ]
)

print("🤖 模型回复：")
print(response.choices[0].message.content)
print(f"\n📊 Token使用：{response.usage.total_tokens}")

In [ ]:
# 2.2 长文本理解（测试128K上下文）
print("📄 测试2：长文本理解\n")

# 生成一段较长的文本
long_text = "人工智能（Artificial Intelligence），英文缩写为AI。" * 50

response = client.chat.completions.create(
    model="glm-4-flash-250414",
    messages=[
        {"role": "system", "content": "你是一个专业的文本分析助手"},
        {"role": "user", "content": f"请总结以下文本的核心观点（50字以内）：\n\n{long_text}"}
    ],
    max_tokens=256
)

print("🤖 总结结果：")
print(response.choices[0].message.content)

## 3. Function Call（工具调用）

In [ ]:
# 3.1 定义工具 - 天气查询
print("🔧 测试1：天气查询工具调用\n")

tools = [
    {
        "type": "function",
        "function": {
            "name": "get_weather",
            "description": "获取指定城市的天气信息",
            "parameters": {
                "type": "object",
                "properties": {
                    "city": {
                        "type": "string",
                        "description": "城市名称，如：北京、上海"
                    },
                    "date": {
                        "type": "string",
                        "description": "日期，格式：YYYY-MM-DD，默认为今天"
                    }
                },
                "required": ["city"]
            }
        }
    }
]

# 询问天气
response = client.chat.completions.create(
    model="glm-4-flash-250414",
    messages=[
        {"role": "user", "content": "今天北京的天气怎么样？"}
    ],
    tools=tools,
    tool_choice="auto"
)

message = response.choices[0].message

# 检查是否触发工具调用
if message.tool_calls:
    print("🔔 模型触发工具调用：")
    for tool_call in message.tool_calls:
        print(f"  工具：{tool_call.function.name}")
        print(f"  参数：{tool_call.function.arguments}")
        
        # 模拟执行工具
        if tool_call.function.name == "get_weather":
            args = json.loads(tool_call.function.arguments)
            city = args.get("city", "未知")
            print(f"\n🌤️ 模拟天气查询结果：{city} 今天晴天，25°C")
else:
    print("💬 模型直接回答：")
    print(message.content)

In [ ]:
# 3.2 多个工具组合
print("\n🔧 测试2：多个工具组合\n")

tools = [
    {
        "type": "function",
        "function": {
            "name": "search",
            "description": "搜索网络信息",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {
                        "type": "string",
                        "description": "搜索关键词"
                    }
                },
                "required": ["query"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "calculator",
            "description": "计算数学表达式",
            "parameters": {
                "type": "object",
                "properties": {
                    "expression": {
                        "type": "string",
                        "description": "数学表达式，如：2+2"
                    }
                },
                "required": ["expression"]
            }
        }
    }
]

response = client.chat.completions.create(
    model="glm-4-flash-250414",
    messages=[
        {"role": "user", "content": "搜索一下北京的人口，然后计算如果每人捐1元，总共能捐多少钱？"}
    ],
    tools=tools,
    tool_choice="auto"
)

message = response.choices[0].message

if message.tool_calls:
    print("🔔 触发的工具调用：")
    for i, tool_call in enumerate(message.tool_calls, 1):
        print(f"\n{i}. 工具：{tool_call.function.name}")
        print(f"   参数：{tool_call.function.arguments}")
else:
    print(message.content)

## 4. 结构化输出（JSON）

In [ ]:
# 4.1 实体抽取 - 从文本中提取结构化信息
print("📝 测试1：实体抽取\n")

sample_text = """
张三，手机号13800138000，于2024年1月15日在北京购买了一台MacBook Pro，
价格为19999元，订单号：ORDER20240115001。
"""

response = client.chat.completions.create(
    model="glm-4-flash-250414",
    messages=[
        {
            "role": "system",
            "content": "你是一个信息抽取助手，请从文本中提取关键信息并以JSON格式输出"
        },
        {
            "role": "user",
            "content": f"请从以下文本中提取实体信息，输出JSON格式（包含：姓名、手机号、日期、地点、商品、价格、订单号）：\n\n{sample_text}"
        }
    ],
    response_format={"type": "json_object"}
)

result = response.choices[0].message.content
print("📊 提取结果：")
try:
    json_result = json.loads(result)
    print(json.dumps(json_result, indent=2, ensure_ascii=False))
except:
    print(result)

In [ ]:
# 4.2 法律文书分析（实体抽取）
print("\n⚖️ 测试2：法律文书分析\n")

legal_text = """
中华人民共和国最高人民法院
指定管辖决定书
（2017）最高法刑辖 19 号
关于被告单位北京盘古氏投资有限公司涉嫌骗取贷款、被告人吕涛等八人涉嫌骗取贷款、
骗购外汇、非国家工作人员受贿、非法拘禁、故意毁坏会计凭证、会计账簿、财务会计报告等犯罪案件，
本院经审查，依照《中华人民共和国刑事诉讼法》第二十六条的规定，决定如下：
指定辽宁省大连市西岗区人民法院依照刑事第一审程序对该案进行审判。
二〇一七年三月十七日
"""

schema = {
    "犯罪客体": {"涉及客体": "string"},
    "犯罪主观要件-罪过形式": {"故意": "string", "过失": "string"},
    "犯罪主观要件": {"犯罪动机": "string", "犯罪目的": "string"},
    "犯罪客观要件": {"犯罪地点": "string", "犯罪行为": "string"},
    "适用法条": "string",
    "判决结果时间": "string"
}

response = client.chat.completions.create(
    model="glm-4-flash-250414",
    messages=[
        {
            "role": "system",
            "content": f"你是一个法律专家，请分析判决书内容并按以下JSON格式输出：{json.dumps(schema, ensure_ascii=False)}"
        },
        {
            "role": "user",
            "content": f"请分析这篇判决书：\n\n{legal_text}"
        }
    ],
    response_format={"type": "json_object"}
)

result = response.choices[0].message.content
print("📊 分析结果：")
try:
    json_result = json.loads(result)
    print(json.dumps(json_result, indent=2, ensure_ascii=False))
except:
    print(result)

## 5. 智能写作

In [ ]:
# 5.1 小红书文案生成
print("✍️ 测试：小红书文案生成\n")

prompt = """
上下文：我想推广公司的新产品。我的公司名为：智谱AI，新产品名为：ChatGLM 大模型，是一款面向大众的 AI 产品。
目标：帮我创建一条小红书平台的帖子，目的是吸引人们点击产品链接进行学习和体验。
风格：参照Dyson等成功公司的宣传风格，同时结合小红书的文案风格。
语调：说服性
受众：AI 产品在小红书上的主要受众是年轻人，活跃在互联网和 AI 领域。
响应：保持小红书帖子简洁而深具影响力，注意要使用emoji表情。
"""

response = client.chat.completions.create(
    model="glm-4-flash-250414",
    messages=[
        {
            "role": "system",
            "content": "你是一个专业的社交媒体文案撰写专家，擅长创作吸引人的小红书内容"
        },
        {
            "role": "user",
            "content": prompt
        }
    ],
    max_tokens=1024,
    temperature=0.8
)

print("📝 生成的小红书文案：\n")
print(response.choices[0].message.content)

## 6. 智能翻译

In [ ]:
# 6.1 文学翻译
print("🌐 测试：文学翻译（莎士比亚）\n")

text_to_translate = """
To be, or not to be: that is the question:
Whether 'tis nobler in the mind to suffer
The slings and arrows of outrageous fortune,
Or to take arms against a sea of troubles
And by opposing end them.
"""

response = client.chat.completions.create(
    model="glm-4-flash-250414",
    messages=[
        {
            "role": "system",
            "content": "你是一个专业的文学翻译家，擅长将英文文学作品翻译成流畅、优雅的中文"
        },
        {
            "role": "user",
            "content": f"请将以下莎士比亚戏剧《哈姆雷特》中的经典独白翻译成中文，保持文学性和优雅：\n\n{text_to_translate}"
        }
    ],
    max_tokens=1024
)

print("📝 原文：")
print(text_to_translate)
print("\n🌐 翻译：")
print(response.choices[0].message.content)

In [ ]:
# 6.2 多语言翻译测试
print("\n🌐 测试：多语言翻译\n")

languages = ["英语", "日语", "法语", "西班牙语"]
text = "人工智能正在改变我们的生活方式"

print(f"原文：{text}\n")

for lang in languages:
    response = client.chat.completions.create(
        model="glm-4-flash-250414",
        messages=[
            {
                "role": "user",
                "content": f"请将以下中文翻译成{lang}：{text}"
            }
        ],
        max_tokens=256
    )
    print(f"{lang}：{response.choices[0].message.content}")

## 7. 流式输出测试

In [ ]:
# 7.1 流式输出
print("🌊 测试：流式输出\n")
print("🤖 模型回复：", end="", flush=True)

response = client.chat.completions.create(
    model="glm-4-flash-250414",
    messages=[
        {"role": "user", "content": "请写一段关于人工智能的简短介绍"}
    ],
    stream=True,
    max_tokens=512
)

for chunk in response:
    if chunk.choices[0].delta.content:
        print(chunk.choices[0].delta.content, end="", flush=True)

print("\n\n✅ 流式输出完成")

## 8. 总结

In [ ]:
print("""
╔════════════════════════════════════════════════════════════════╗
║               GLM-4-Flash-250414 测试总结                      ║
╠════════════════════════════════════════════════════════════════╣
║ 模型定位：智谱首个免费大模型 API                                 ║
╠════════════════════════════════════════════════════════════════╣
║ 核心能力：                                                      ║
║   ✅ 流式输出 - 实时响应                                        ║
║   ✅ Function Call - 强大的工具调用能力                          ║
║   ✅ 上下文缓存 - 优化长对话性能                                 ║
║   ✅ 结构化输出 - JSON 格式输出                                 ║
║   ✅ MCP - 外部工具调用                                         ║
║   ❌ 思考模式 - 不支持深度思考                                   ║
╠════════════════════════════════════════════════════════════════╣
║ 功能特色：                                                      ║
║   • 超长上下文 - 128K（相当于 300 页书籍）                       ║
║   • 多语言支持 - 支持 26 种语言                                 ║
║   • 网页检索 - 支持外部工具调用获取网络信息                      ║
╠════════════════════════════════════════════════════════════════╣
║ 适用场景：                                                      ║
║   • 智能写作 - 小红书文案、商业文案、文学创作                    ║
║   • 智能翻译 - 多语言互译、文学翻译                              ║
║   • 实体抽取 - 从非结构化文本中提取结构化数据                    ║
║   • 工具调用 - 天气查询、搜索、计算等                            ║
║   • 长文本处理 - 摘要生成、文档分析                              ║
╠════════════════════════════════════════════════════════════════╣
║ 与 GLM-4.7-Flash 的区别：                                      ║
║   • GLM-4-Flash-250414：不支持思考模式，但功能全面              ║
║   • GLM-4.7-Flash：支持思考模式，上下文更大（200K）             ║
╚════════════════════════════════════════════════════════════════╝
""")